In [ ]:
# Exercise sheet 3 with he file Potter.zip.  ”Harry Potter”-books. 

# Task 1
# In Moodle you will find the file Potter.zip. Unpack it. It contains 7 txt-files, each containing the text one of the ”Harry Potter”-books. Load those txt-files into your console.

In [1]:
# option 1

import zipfile
import os

# Define the path to the zip file and extraction directory
zip_file_path = '/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data/Potter.zip'
extraction_dir = '/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data/Potter'

# Unpack the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

# Load all txt files into a list
book_texts = []
for filename in os.listdir(extraction_dir):
    if filename.endswith('.txt'):
        with open(os.path.join(extraction_dir, filename), 'r', encoding='utf-8') as file:
            book_texts.append(file.read())

# Print loaded book texts (or their lengths for brevity)
print([len(text) for text in book_texts])  # Output lengths of each book's text

[1187250, 1059022, 531708, 474429, 1608763, 676978, 1227024]


In [2]:
# option 2

import os
import zipfile
import glob

# Define paths
data_folder = "/Users/oayanwale/Downloads/NLP_Exercise_24_25/Data/"
zip_path = os.path.join(data_folder, "Potter.zip")
extract_folder = os.path.join(data_folder, "PotterBooks")

# Unzip Potter.zip
if not os.path.exists(extract_folder):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)

# Load the text files
book_files = glob.glob(os.path.join(extract_folder, "*.txt"))

# Read books into a dictionary
books = {}
for file_path in book_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        books[os.path.basename(file_path)] = f.readlines()

print(f"Loaded {len(books)} books.")


Loaded 7 books.


# Task 2
# To compare the books, we must know, which book it is we are looking at. Each file contains one particular line for every page in the book:
# Page | page number book name - J.K. Rowling
# Use regular expressions to automatically detect the name of the book from the texts.


In [9]:
#option 1

import re

book_names_dict = {}
for filename, text in zip(os.listdir(extraction_dir), book_texts):
    match = re.search(r"(.+)- J\.K\. Rowling", text)
    if match:
        # Strip leading/trailing whitespace and add to dictionary
        book_name = match.group(1).strip()
        book_names_dict[filename] = book_name

print("Detected Book Names Dictionary:", book_names_dict)

Detected Book Names Dictionary: {'file2.txt': 'Harry Potter and the Goblet of Fire', 'file3.txt': 'Page | 2 Harry Potter and the Half Blood Prince', 'file1.txt': 'Page | 2 Harry Potter and the Chamber of Secrets', 'file4.txt': 'Page | 2 Harry Potter and the Philosophers Stone', 'file5.txt': 'Page | 2 Harry Potter and the Order of the Phoenix', 'file7.txt': 'Page | 2 Harry Potter and the Prisoner of Azkaban', 'file6.txt': 'Page | 2 Harry Potter and the Deathly Hallows'}


In [12]:
# option 1 adjusted
import re

book_names_dict = {}
for filename, text in zip(os.listdir(extraction_dir), book_texts):
    # Adjust regex to capture clean book names without using look-behind
    match = re.search(r'Page \| \d+ (.+?) - J\.K\. Rowling', text)
    if match:
        # Strip leading/trailing whitespace and add to dictionary
        book_name = match.group(1).strip()
        book_names_dict[filename] = book_name

print("Detected Book Names Dictionary:", book_names_dict)

Detected Book Names Dictionary: {'file2.txt': 'Harry Potter and the Goblet of Fire', 'file3.txt': 'Harry Potter and the Half Blood Prince', 'file1.txt': 'Harry Potter and the Chamber of Secrets', 'file4.txt': 'Harry Potter and the Philosophers Stone', 'file5.txt': 'Harry Potter and the Order of the Phoenix', 'file7.txt': 'Harry Potter and the Prisoner of Azkaban', 'file6.txt': 'Harry Potter and the Deathly Hallows'}


In [4]:
# option 2 but better to use option 1 adjusted
import re

def extract_book_title(text_lines):
    for line in text_lines:
        match = re.search(r"(.+)- J\.K\. Rowling", line)
        if match:
            return match.group(1).strip()
    return "Unknown Title"

book_titles = {file: extract_book_title(content) for file, content in books.items()}
print(book_titles)


{'file2.txt': 'Harry Potter and the Goblet of Fire', 'file3.txt': 'Page | 2 Harry Potter and the Half Blood Prince', 'file1.txt': 'Page | 2 Harry Potter and the Chamber of Secrets', 'file4.txt': 'Page | 2 Harry Potter and the Philosophers Stone', 'file5.txt': 'Page | 2 Harry Potter and the Order of the Phoenix', 'file7.txt': 'Page | 2 Harry Potter and the Prisoner of Azkaban', 'file6.txt': 'Page | 2 Harry Potter and the Deathly Hallows'}



# Task 3
# The texts in the the txt-files are not ”clean” yet. To analyze them properly, we need to do additional preprocessing steps.
# • Remove the page indicator from the texts. That is, remove all lines that have the form mentioned in task 2
# • Trim the start of the document until the first chapter starts.
# • Remove the headers of all chapters. These are written in CAPS (all letters are capitalized).
# Detect this using regular expressions.
# • Replace all line breaks (”\n”) with a whitespace (” ”).
# The result should be a list of 7 large stings, one for each book.


In [24]:
import re

def clean_book_text(text):
    # Step 1: Remove page indicators using regex
    cleaned_text = re.sub(r'Page \| \d+ .+? - J\.K\. Rowling\n?', '', text)

    ## Step 2: Trim until first chapter starts (assuming chapters start with "CHAPTER")
    cleaned_text = cleaned_text.split('CHAPTER', 1)[-1] if 'CHAPTER' in cleaned_text else cleaned_text

    # Step 3: Remove chapter headers (all caps)
    cleaned_text = re.sub(r'\n[A-Z\s]+\n?', ' ', cleaned_text)

    # Step 4: Replace line breaks with whitespace
    cleaned_text = re.sub(r'\n+', ' ', cleaned_text).strip()

    return cleaned_text

# Clean all books
cleaned_books = [clean_book_text(text) for text in book_texts]

# Print lengths of cleaned books to verify processing
print("Cleaned Books Lengths:", [len(book) for book in cleaned_books])

Cleaned Books Lengths: [1145446, 997862, 497716, 443794, 1578245, 632635, 1157662]


In [34]:
def clean_book_text(text_lines):
    # Remove page indicators
    text_lines = [line for line in text_lines if not re.match(r"Page \| \d+", line)]
    
    # Find where the first chapter starts
    start_idx = next((i for i, line in enumerate(text_lines) if "CHAPTER" in line.upper()), 0)
    text_lines = text_lines[start_idx:]

    # Remove chapter headers (all caps words)
    text_lines = [line for line in text_lines if not re.fullmatch(r"[A-Z\s]+", line.strip())]

    # Merge lines into a single string
    return " ".join(line.strip() for line in text_lines if line.strip())

cleaned_books = {title: clean_book_text(content) for title, content in books.items()}


# Task 4
# Apply elementary preprocessing steps in any order you prefer: punctuation- and number re- moval, lower casing, lemmatization/stemming and stop word removal. The result should be a list of lists. Each inner list represents a book as a list of words.


In [25]:
# option 1

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Download necessary NLTK resources if not already done.
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_book(book):
    # Step 1: Tokenize the text into words, removing punctuation and numbers
    words = re.findall(r'\b\w+\b', book.lower())
    
    # Step 2: Lemmatization and stop word removal 
    processed_words = [
        lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return processed_words

# Process all cleaned books
processed_books = [preprocess_book(book) for book in cleaned_books]

# Print a sample of processed books (first 10 words of each)
print("Processed Books Sample:", [processed[:10] for processed in processed_books])

[nltk_data] Downloading package punkt to /Users/oayanwale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Processed Books Sample: [['villager', 'little', 'hangleton', 'still', 'called', 'iddle', 'house', 'even', 'though', 'many'], ['j', 'nearing', 'midnight', 'prime', 'minister', 'sitting', 'alone', 'office', 'reading', 'long'], ['k', 'r', 'w', 'l', 'n', 'g', 'ot', 'first', 'time', 'argument'], ['r', 'mr', 'dursley', 'number', 'four', 'privet', 'drive', 'proud', 'say', 'perfectly'], ['harry', 'hottest', 'day', 'summer', 'far', 'drawing', 'close', 'drowsy', 'silence', 'lay'], ['arry', 'potter', 'highly', 'unusual', 'boy', 'many', 'way', 'one', 'thing', 'hated'], ['two', 'men', 'appeared', 'nowhere', 'yard', 'apart', 'narrow', 'moonlit', 'lane', 'second']]


In [27]:
#option 1b

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

# Ensure necessary resources are available
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stop words and stemmer/lemmatizer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize words
    words = word_tokenize(text)

    # Remove punctuation &; numbers, convert to lowercase
    words = [word.lower() for word in words if word.isalpha()]

    # Apply stemming (or lemmatization based on preference)
    # Uncomment one of the following lines based on your choice:
    # processed_words = [stemmer.stem(word) for word in words]
    processed_words = [lemmatizer.lemmatize(word) for word in words]

    # Remove stopwords
    processed_words = [word for word in processed_words if word not in stop_words]

    return processed_words

# Process all cleaned books using the new preprocessing function
processed_books = [preprocess_text(book) for book in cleaned_books]

# Print a sample of processed books (first 10 words of each)
print("Processed Books Sample:", [processed[:10] for processed in processed_books])

[nltk_data] Downloading package punkt to /Users/oayanwale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Processed Books Sample: [['villager', 'little', 'hangleton', 'still', 'called', 'iddle', 'house', 'even', 'though', 'many'], ['j', 'wa', 'nearing', 'midnight', 'prime', 'minister', 'wa', 'sitting', 'alone', 'office'], ['k', 'r', 'w', 'l', 'n', 'g', 'ot', 'first', 'time', 'argument'], ['dursley', 'number', 'four', 'privet', 'drive', 'proud', 'say', 'perfectly', 'normal', 'thank'], ['harry', 'hottest', 'day', 'summer', 'far', 'wa', 'drawing', 'close', 'drowsy', 'silence'], ['arry', 'potter', 'wa', 'highly', 'unusual', 'boy', 'many', 'way', 'one', 'thing'], ['two', 'men', 'appeared', 'nowhere', 'yard', 'apart', 'narrow', 'moonlit', 'lane', 'second']]


Explanation of Each Step:
Tokenization: We use a regex pattern (\b\w+\b) to find all word-like sequences while ignoring punctuation and numbers.
Lowercasing: Converting everything to lowercase ensures uniformity when analyzing the text.
Lemmatization: This reduces words to their base form (e.g., "running" becomes "run").
Stop Word Removal: Common English stop words (like "and", "the", etc.) are filtered out to focus on more meaningful terms.

In [52]:
# option 2 the best here 

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Ensure necessary resources are available
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize words
    words = word_tokenize(text)

    # Remove punctuation & numbers, convert to lowercase
    words = [word.lower() for word in words if word.isalpha()]

    # Apply stemming and lemmatization
    #stemmed_words = [stemmer.stem(word) for word in words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Remove stopwords
    processed_words = [word for word in lemmatized_words if word not in stop_words]

    return processed_words

preprocessed_books = {title: preprocess_text(content) for title, content in cleaned_books.items()}

# Print a sample of processed books (first 10 words of each)
print("Processed Books Sample:", [processed[:10] for processed in processed_books])



[nltk_data] Downloading package punkt to /Users/oayanwale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oayanwale/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Processed Books Sample: [['villager', 'little', 'hangleton', 'still', 'called', 'iddle', 'house', 'even', 'though', 'many'], ['j', 'wa', 'nearing', 'midnight', 'prime', 'minister', 'wa', 'sitting', 'alone', 'office'], ['k', 'r', 'w', 'l', 'n', 'g', 'ot', 'first', 'time', 'argument'], ['dursley', 'number', 'four', 'privet', 'drive', 'proud', 'say', 'perfectly', 'normal', 'thank'], ['harry', 'hottest', 'day', 'summer', 'far', 'wa', 'drawing', 'close', 'drowsy', 'silence'], ['arry', 'potter', 'wa', 'highly', 'unusual', 'boy', 'many', 'way', 'one', 'thing'], ['two', 'men', 'appeared', 'nowhere', 'yard', 'apart', 'narrow', 'moonlit', 'lane', 'second']]


In [53]:
# Print a sample from the first book (first 50 words)
sample_book_title = list(preprocessed_books.keys())[0]  # Get the first book title
sample_words = preprocessed_books[sample_book_title][:50]  # First 50 words

print(f"Sample processed text from '{sample_book_title}':")
print(" ".join(sample_words))


Sample processed text from 'file2.txt':
villager little hangleton still called riddle house even though many year since riddle family lived stood hill overlooking village window boarded tile missing roof ivy spreading unchecked face manor easily largest grandest building mile around riddle house wa damp derelict unoccupied little hangletons agreed old house wa half century ago


In [54]:
# Some words like "wa" (past tense was) are not removed by default. Let's add custom stop words:

custom_stop_words = set(stop_words).union({"wa", "ha", "said"})  # Add more as needed

def preprocess_text(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha()]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return [word for word in lemmatized_words if word not in custom_stop_words]

preprocessed_books = {title: preprocess_text(content) for title, content in cleaned_books.items()}


In [55]:
sample_book_title = list(preprocessed_books.keys())[0]
sample_words = preprocessed_books[sample_book_title][:50]

print(f"Improved sample processed text from '{sample_book_title}':")
print(" ".join(sample_words))


Improved sample processed text from 'file2.txt':
villager little hangleton still called riddle house even though many year since riddle family lived stood hill overlooking village window boarded tile missing roof ivy spreading unchecked face manor easily largest grandest building mile around riddle house damp derelict unoccupied little hangletons agreed old house half century ago something strange


# Task 5
# Calculate the tfidf for your corpus. Return the words with the highest tfidf for each of the 7 books. # Does the result give you an idea of what the books are about? If not, why?

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine processed words into a single string per book
combined_books = {title: " ".join(content) for title, content in preprocessed_books.items()}

# Create a TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the combined text data
tfidf_matrix = vectorizer.fit_transform(combined_books.values())

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Extract top n terms with highest tf-idf score for each book
top_n = 10  # Define how many top terms you want to retrieve
tfidf_results = {}

for i, title in enumerate(combined_books.keys()):
    # Get sorted indices of tf-idf scores in descending order
    sorted_indices = tfidf_matrix[i].toarray().flatten().argsort()[::-1]
    
    # Retrieve top n terms and their corresponding scores
    top_terms_with_scores = [(feature_names[idx], tfidf_matrix[i, idx]) 
                             for idx in sorted_indices[:top_n]]
    
    # Store results in dictionary
    tfidf_results[title] = top_terms_with_scores

# Print out top n terms with their TF-IDF scores for each book
for title, terms in tfidf_results.items():
    print(f"\nTop {top_n} TF-IDF Words for {title}:")
    for term, score in terms:
        print(f"{term}: {score:.4f}")


Top 10 TF-IDF Words for file2.txt:
harry: 0.6935
ron: 0.2017
hermione: 0.1680
potter: 0.1268
rowling: 0.1215
dumbledore: 0.1150
back: 0.1144
fire: 0.1065
bagman: 0.1044
one: 0.0976

Top 10 TF-IDF Words for file3.txt:
harry: 0.6782
dumbledore: 0.2788
slughorn: 0.1940
ron: 0.1793
hermione: 0.1259
could: 0.1223
would: 0.1212
snape: 0.1021
malfoy: 0.0984
back: 0.0953

Top 10 TF-IDF Words for file1.txt:
harry: 0.6898
ron: 0.2973
hermione: 0.1597
lockhart: 0.1541
riddle: 0.1182
back: 0.1037
one: 0.1019
malfoy: 0.0971
could: 0.0929
professor: 0.0894

Top 10 TF-IDF Words for file4.txt:
harry: 0.6642
ron: 0.2140
hagrid: 0.1863
hermione: 0.1345
one: 0.1314
back: 0.1299
got: 0.1037
get: 0.1022
know: 0.1017
could: 0.0997

Top 10 TF-IDF Words for file5.txt:
harry: 0.6414
hermione: 0.2363
ron: 0.2105
umbridge: 0.1984
back: 0.1323
professor: 0.1148
well: 0.1141
sirius: 0.1092
could: 0.1043
dumbledore: 0.1010

Top 10 TF-IDF Words for file7.txt:
harry: 0.6595
ron: 0.2659
hermione: 0.2443
lupin: 0.1990

Great! The TF-IDF results obtained provide a lot of insight into the content of each book. Here's a summary of what you can interpret from these results:

### Observations:
1. **Frequent Character Names**: 
   - **Harry, Ron, Hermione**: These three characters consistently appear among the top terms across all books, indicating their central roles in the story.
   - Other notable characters like **Dumbledore**, **Snape**, and **Hagrid** also appear frequently, which aligns with their importance in the series.

2. **Contextual Words**:
   - Words like **"back"**, **"could"**, and **"one"** are common across multiple books but may not carry specific thematic weight on their own.
   - Specific terms related to events or elements in the story (e.g., **"umbridge," "lockhart," "lupin," "bagman,"**) suggest key plot points or themes relevant to those particular books.

3. **Unique Terms per Book**: 
   - Each book has unique words that reflect its specific plot elements or character arcs (e.g., "umbridge" in file5.txt indicates her significance in that particular book).

### Analysis of Content:
- The presence of character names as high-TFIDF terms suggests that these books focus heavily on interpersonal relationships and character development.
- If certain expected themes or important concepts are missing from your results, consider whether stop words or lemmatization may have inadvertently filtered them out.

### Conclusion:
The TF-IDF analysis gives you a good idea of what each book is about by highlighting significant characters and plot-related terms. 
It shows how much emphasis is placed on certain characters throughout the series and can help identify key themes present in each installment.


In [58]:
# option 2

from sklearn.feature_extraction.text import TfidfVectorizer

# Convert each book into a full string for TF-IDF processing
book_texts = [" ".join(words) for words in preprocessed_books.values()]

# Apply TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Limit to 5000 most important words
tfidf_matrix = vectorizer.fit_transform(book_texts)
feature_names = vectorizer.get_feature_names_out()

# Function to get the highest TF-IDF words per book
def get_top_tfidf_words(tfidf_matrix, feature_names, top_n=10):
    top_words = {}
    for book_idx, title in enumerate(preprocessed_books.keys()):
        row = tfidf_matrix[book_idx].toarray()[0]
        top_indices = row.argsort()[-top_n:][::-1]  # Get top N words with highest TF-IDF
        top_words[title] = [feature_names[i] for i in top_indices]
    return top_words

# Get top TF-IDF words
top_tfidf_words = get_top_tfidf_words(tfidf_matrix, feature_names)

# Print results
for book, words in top_tfidf_words.items():
    print(f"\nTop TF-IDF words for '{book}':")
    print(", ".join(words))




Top TF-IDF words for 'file2.txt':
harry, ron, hermione, potter, rowling, dumbledore, back, fire, bagman, one

Top TF-IDF words for 'file3.txt':
harry, dumbledore, slughorn, ron, hermione, could, would, snape, malfoy, back

Top TF-IDF words for 'file1.txt':
harry, ron, hermione, lockhart, riddle, back, one, malfoy, could, professor

Top TF-IDF words for 'file4.txt':
harry, ron, hagrid, hermione, one, back, got, get, know, could

Top TF-IDF words for 'file5.txt':
harry, hermione, ron, umbridge, back, professor, well, sirius, could, dumbledore

Top TF-IDF words for 'file7.txt':
harry, ron, hermione, lupin, black, professor, back, snape, one, hagrid

Top TF-IDF words for 'file6.txt':
harry, hermione, ron, could, wand, dumbledore, back, know, like, one
